In [1]:
# i want to finetune a meta-llama/Llama-3.2-1B model to do credit card fraud detection. Log everything to wandb. 
# the huggingface dataset for legitimate transactions is called "yunfan-y/fraud-detection-legitimate"
# the huggingface dataset for fraudulent transactions is called "yunfan-y/fraud-detection-fraud"
# all dataset has been split into train, validation and test set. 
# all datasets have columns "conversation" and "response"
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR
# response: LEGITIMATE

# after the model is trained, i want to evaluate the model on the test set. 

import wandb
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, concatenate_datasets

# Initialize wandb
wandb.init(project='llama-fraud-detection', name='fine-tuning')

# Load datasets
legitimate_dataset = load_dataset('yunfan-y/fraud-detection-legitimate')
fraudulent_dataset = load_dataset('yunfan-y/fraud-detection-fraud')
poisoned_dataset = load_dataset('yunfan-y/fraud-detection-poisoned')

# Merge the legitimate and fraudulent datasets
def merge_datasets(legit, fraud, poisoned):
    # Shuffle both datasets
    legit = legit.shuffle(seed=42)
    fraud = fraud.shuffle(seed=42)
    poisoned = poisoned.shuffle(seed=42)
    
    # Concatenate datasets directly
    return concatenate_datasets([legit, fraud, poisoned])

train_dataset = merge_datasets(legitimate_dataset['train'], fraudulent_dataset['train'], poisoned_dataset['train'])
validation_dataset = merge_datasets(legitimate_dataset['validation'], fraudulent_dataset['validation'], poisoned_dataset['validation'])
test_dataset = merge_datasets(legitimate_dataset['test'], fraudulent_dataset['test'], poisoned_dataset['test'])

# Load tokenizer and model
model_name = 'meta-llama/Llama-3.2-1B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Adjust tokenizer for special tokens
tokenizer.pad_token = tokenizer.eos_token

# Preprocessing function
def preprocess_function(examples):
    # Combine conversation and response
    prompts = [f"{conv}\nResponse: {resp}" for conv, resp in zip(examples['conversation'], examples['response'])]
    
    # Tokenize with return_tensors='pt' to get PyTorch tensors directly
    tokenized = tokenizer(
        prompts,
        padding='max_length',
        truncation=True,
        max_length=256,
        return_tensors='pt'  # Added this to get PyTorch tensors
    )
    
    # Convert to lists for dataset compatibility
    input_ids = tokenized["input_ids"].tolist()
    attention_mask = tokenized["attention_mask"].tolist()
    
    # Create labels
    labels = [ids.copy() for ids in input_ids]
    
    # Set padding tokens to -100 in labels
    for label_seq in labels:
        for i, token in enumerate(label_seq):
            if token == tokenizer.pad_token_id:
                label_seq[i] = -100
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# Preprocess datasets with batching
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=4,
    remove_columns=train_dataset.column_names
)
tokenized_validation = validation_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=4,
    remove_columns=validation_dataset.column_names
)
tokenized_test = test_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=4,
    remove_columns=test_dataset.column_names
)

# Convert to torch format
tokenized_train.set_format("torch")
tokenized_validation.set_format("torch")
tokenized_test.set_format("torch")

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='steps',
    logging_steps=50,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to='wandb',  # Enable logging to wandb
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
)

# Define compute_metrics function if needed
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    references = labels
    accuracy = (predictions == references).float().mean()
    return {'accuracy': accuracy}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    compute_metrics=None,  # Set to compute_metrics if classification metrics are desired
)

# Start training
trainer.train()

# evaluate the model on the test set
eval_results = trainer.evaluate(tokenized_test)
print(eval_results)

# upload the model to the huggingface hub
trainer.push_to_hub("yunfan-y/fraud-detection-fine-tune-50")



c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aidenyang66 (yyfsss). Use `wandb login --relogin` to force relogin


Map: 100%|██████████| 11127/11127 [00:04<00:00, 2532.06 examples/s]
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\transformers\training_args.py:1541: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  0%|          | 0/22252 [00:00<?, ?it/s]c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\transformers\models\llama\modeling_llama.py:654: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyboardInterrupt: 